In [4]:
import os, re, json
import pdfplumber
from tqdm import tqdm

In [ ]:
pip install langchain_text_splitters

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\rasso\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ---------- CONFIG ----------
PDF_DIR = "regulations_texts"           # folder with your PDFs
OUTPUT_JSONL = "eu_laws_chunks.jsonl"
# ----------------------------


def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() or "" for page in pdf.pages)
    # Basic cleanup
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s{2,}', ' ', text)
    return text.strip()


def chunk_eu_law(text, doc_title):
    """Chunk EU regulations/directives by recitals and articles."""
    chunks = []

    # --- 1️⃣ Split recitals ---
    recitals = re.findall(r"\(\d+\).*?(?=\(\d+\)|HAVE ADOPTED|TITLE I|Article\s+1)", text, re.S)
    for r in recitals:
        num = re.match(r"\((\d+)\)", r)
        if num:
            chunks.append({
                "section": "Recital",
                "number": int(num.group(1)),
                "text": r.strip(),
                "document_title": doc_title
            })

    # --- 2️⃣ Split articles ---
    # find all "Article X" sections
    article_blocks = re.split(r"(?=Article\s+\d+)", text)
    for art in article_blocks:
        match = re.match(r"Article\s+(\d+)", art)
        if match:
            art_num = int(match.group(1))
            chunks.append({
                "section": "Article",
                "article_number": art_num,
                "text": art.strip(),
                "document_title": doc_title
            })

    # --- 3️⃣ Fallback chunking for very long texts ---
    splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
    final_chunks = []
    for c in chunks:
        smalls = splitter.split_text(c["text"])
        for i, s in enumerate(smalls):
            meta = c.copy()
            meta["chunk_id"] = f"{c.get('section')}_{c.get('number', c.get('article_number', 'x'))}_{i}"
            meta["text"] = s
            final_chunks.append(meta)

    return final_chunks


def process_all_pdfs(pdf_dir):
    """Process all PDFs in a folder and return chunks."""
    all_chunks = []
    for filename in tqdm(os.listdir(pdf_dir)):
        if filename.lower().endswith(".pdf"):
            path = os.path.join(pdf_dir, filename)
            print(f"📄 Processing {filename}...")
            text = extract_text_from_pdf(path)
            chunks = chunk_eu_law(text, filename.replace(".pdf", ""))
            all_chunks.extend(chunks)
    return all_chunks


# ---------- Run the pipeline ----------
all_chunks = process_all_pdfs(PDF_DIR)

print(f"\n✅ Total chunks created: {len(all_chunks)}")

# Save as JSONL
with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
    for c in all_chunks:
        json.dump(c, f, ensure_ascii=False)
        f.write("\n")

print(f"💾 Chunks saved to {OUTPUT_JSONL}")

  0%|          | 0/9 [00:00<?, ?it/s]

📄 Processing AI_ACT.pdf...


 11%|█         | 1/9 [00:51<06:50, 51.35s/it]

📄 Processing AudiovisualServices_SMA.pdf...


 22%|██▏       | 2/9 [01:02<03:13, 27.63s/it]

...

📄 Processing Intellectual_Property.pdf...


 89%|████████▉ | 8/9 [03:29<00:30, 30.14s/it]

📄 Processing NIS2_Cybersecu.pdf...


100%|██████████| 9/9 [03:53<00:00, 25.98s/it]



✅ Total chunks created: 6189
💾 Chunks saved to eu_laws_chunks.jsonl
